# 8.4 Constraints

The simplest kind of constraint declaration begins with the keywords `subject to`, a
name, and a colon. Even the `subject to` is optional; AMPL assumes that any declaration
not beginning with a keyword is a constraint. Following the colon is an algebraic
description of the constraint, in terms of previously defined sets, parameters, and
variables.

Thus, in the production model introduced in
[Figure 1-4a](../01/tut_1_4.ipynb#fig-1-4a),
we have the following constraint imposed by limited processing time:

```ampl
subject to Time:
  sum {p in PROD} (1 / rate[p]) * Make[p] <= avail;
```

The name of a constraint, like the name of an objective, is not used elsewhere in an
algebraic model, though it figures in alternative *columnwise* formulations (Chapter 16)
and is used in the AMPL command environment to specify the constraint's dual value
and other associated quantities (Chapter 14) xTODO.

Most of the constraints in large linear programming models are defined as indexed
collections, by giving an indexing expression after the constraint name. The constraint
`Time`, for example, is generalized in subsequent examples to say that the production
time may not exceed the time available in each processing stage `s`
([Figure 1-6](../01/tut_1_6.ipynb#fig-1-6)):

```ampl
subject to Time {s in STAGE}:
  sum {p in PROD} (1 / rate[p,s]) * Make[p] <= avail[s];
```

or in each week `t`
([Figure 4-4](../04/4_3_a_model_of_production_and_transportation.ipynb#fig-4-4)):

```ampl
subject to Time {t in 1..T}:
  sum {p in PROD} (1 / rate[p]) * Make[p,t] <= avail[t];
```

Another constraint from the latter example says that production, sales, and inventories
must balance for each product `p` in each week `t`:

```ampl
subject to Balance {p in PROD, t in 1..T}:
  Make[p,t] + Inv[p,t-1] = Sell[p,t] + Inv[p,t];
```

A constraint declaration can specify any valid indexing expression, which defines a set
(as explained in Chapters [5](../05/05.md) and [6](../06/06.md)); there is one constraint for each member of this set.
The constraint name can be subscripted, so that `Time[1]` or `Balance[p,t+1]` refers
to a particular constraint from an indexed collection.

The indexing expression in a constraint declaration should specify a dummy index
(such as `s`, `t`, or `p` in the preceding examples) for each dimension of the indexing
set. When the constraint corresponding to a particular indexing-set member is processed
by AMPL, the dummy indices take their values from that member.

This use of dummy indices is what permits a single constraint expression to represent
many constraints; the indexing expression is AMPL's translation of a phrase such as
"for all products `p` and weeks `t = 1` to `T`" that might be seen in an algebraic
statement of the model.

By using more complex indexing expressions, you can specify more precisely the
constraints to be included in a model. Consider, for example, the following variation
on the production-time constraint:

```ampl
subject to Time {t in 1..T: avail[t] > 0}:
  sum {p in PROD} (1 / rate[p]) * Make[p,t] <= avail[t];
```

This says that if `avail[t]` is specified as zero in the data for any week `t`, it is to be
interpreted as meaning *no constraint on time available in week `t`*, rather than a
limit of zero.

In the simpler case where there is just one `Time` constraint not indexed over weeks,
you can specify an analogous conditional definition as follows:

```ampl
subject to Time {if avail > 0}:
  sum {p in PROD} (1 / rate[p]) * Make[p] <= avail;
```

The pseudo-indexing expression `{if avail > 0}` causes one constraint, named `Time`,
to be generated if the condition `avail > 0` is true, and no constraint at all to be
generated if the condition is false. (The same notation can be used to conditionally
define other model components.)

AMPL's algebraic description of a constraint may consist of any two linear expressions
separated by an equality or inequality operator:

```
linear-expr <= linear-expr
linear-expr =  linear-expr
linear-expr >= linear-expr
```

While it is customary in mathematical descriptions of linear programming to place all
terms containing variables on the left-hand side of the operator and all other terms on
the right (as in constraint `Time`), AMPL imposes no such requirement (as seen in
constraint `Balance`).

Convenience and readability should determine what terms you place on each side of the
operator. AMPL takes care of canonicalizing constraints, such as by combining linear
terms involving the same variable and moving variables from one side of a constraint to
the other. The `expand` command described in
[Section 1.4](../01/tut_1_4.ipynb)
shows the canonical forms of the constraints.

AMPL also allows double-inequality constraints such as the following from the diet
model of
[Figure 2-1](../02/2_2_an_AMPL_model_for_the_diet_problem.ipynb#fig-2-1):

```ampl
subject to Diet {i in NUTR}:
  n_min[i] <= sum {j in FOOD} amt[i,j] * Buy[j] <= n_max[i];
```

This says that the middle expression—the amount of nutrient `i` supplied by all foods—
must be greater than or equal to `n_min[i]` and less than or equal to `n_max[i]`.

The permissible forms for a constraint of this kind are

```
const-expr <= linear-expr <= const-expr
const-expr >= linear-expr >= const-expr
```

where each `const-expr` must contain no variables. The effect is to give upper and lower
bounds on the value of the `linear-expr`. If your model requires variables in the
left-hand or right-hand constant expression, you must define two different constraints
in separate declarations.

For most applications of linear programming, you need not worry about the form of the
constraints. If you simply write the constraints in the most convenient way, they will
be recognized as proper linear constraints according to the rules in this chapter. (xTODO talk about MP?)

There do exist situations, however, in which your choice of formulation determines
whether AMPL recognizes your model as linear. Suppose we want to constrain the
production model so that no product `p` may represent more than a certain fraction of
total production. Let `max_frac` denote this fraction. A direct formulation would be

```ampl
subject to Limit {p in PROD}:
  Make[p] / sum {q in PROD} Make[q] <= max_frac;
```

This is not linear to AMPL, because its left-hand expression contains a division by
a sum of variables. But if we rewrite it as

```ampl
subject to Limit {p in PROD}:
  Make[p] <= max_frac * sum {q in PROD} Make[q];
```

then AMPL does recognize it as linear.

AMPL simplifies constraints as it prepares the model and data for handing to a solver.
For example, it may eliminate variables fixed at a value, combine single-variable
constraints with simple bounds on variables, or drop constraints implied by other
constraints.

You can normally ignore this *presolve* phase, but there are ways to observe its effects
and modify its actions, as explained in
Section 14.1 xTODO.